In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Loading the 12 split CSVs and combining back into a single dataframe

- As the original CSV has been split into 12, only the first file contains the proper column names.
- Therefore, we need to ensure that files 2 through to 12 are read in a way where their first row isn't treated as the column names.

In [ ]:
# Load CSV, need to go up one folder level then back down
df = pd.read_csv('Raw Data/all_energy_statistics1.csv')
column_names = df.columns.to_list()

In [ ]:
# Iterating through every subsquent CSV and appending to first CSV
file_number = 2
for i in np.arange(11):
    # print(f'About to load file number {file_number}') # for debugging/ sanity check
    df_ = pd.read_csv(f'Raw Data/all_energy_statistics{file_number}.csv',header=None)
    df_.columns = column_names
    df = df.append(df_)
    file_number+=1

In [ ]:
df

In [5]:
# Checking that the CSV splitting & rejoining process worked correctly - does my newly rejoined/ appended dataframe match shape of dataframe from Kaggle kernel?
row_num_kaggle = 1189482 # https://www.kaggle.com/alexanderklarge/checking-out-data-set-for-seeyoudata-project
row_num_here = df.shape[0]
assert row_num_kaggle == row_num_here, 'Dataframes don\'t match!'

## Cleaning the dataframe

- First of all, I'd like to take the commodity_transaction column and turn it into a separate column per commodity type

In [6]:
df.head(3)

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates


In [7]:
# Creating a groupby
# Problem here is losing string columns i.e. unit, but I suppose I can rejoin them on somehow later
cols_of_interest = ['country_or_area','year','commodity_transaction','quantity']
df_groupby = df[cols_of_interest].groupby(['country_or_area','year','commodity_transaction']).sum()
df_groupby.head(3)

quantity
country_or_area year commodity_transaction                                       
Afghanistan     1990 Aviation gasoline - Consumption by transport             3.0
                     Aviation gasoline - Consumption in domestic avi...       3.0
                     Aviation gasoline - Final consumption                    3.0

In [8]:
# Using the mystical unstack method to magically turn the commodity_transaction column into a column per commodity_transaction type!
df_groupby = df_groupby.unstack()

In [9]:
df_groupby.head(3)

quantity  \
commodity_transaction Additives and Oxygenates - Exports   
country_or_area year                                       
Afghanistan     1990                                 NaN   
                1991                                 NaN   
                1992                                 NaN   

                                                          \
commodity_transaction Additives and Oxygenates - Imports   
country_or_area year                                       
Afghanistan     1990                                 NaN   
                1991                                 NaN   
                1992                                 NaN   

                                                             \
commodity_transaction Additives and Oxygenates - Production   
country_or_area year                                          
Afghanistan     1990                                    NaN   
                1991                                    NaN   
                1992                                    NaN   

                                                                              \
commodity_transaction Additives and Oxygenates - Receipts from other sources   
country_or_area year                                                           
Afghanistan     1990                                                 NaN       
                1991                                                 NaN       
                1992                                                 NaN       

                                                                \
commodity_transaction Additives and Oxygenates - Stock changes   
country_or_area year                                             
Afghanistan     1990                                       NaN   
                1991                                       NaN   
                1992                                       NaN   

                                                                      \
commodity_transaction Additives and Oxygenates - Total energy supply   
country_or_area year                                                   
Afghanistan     1990                                             NaN   
                1991                                             NaN   
                1992                                             NaN   

                                                                 \
commodity_transaction Additives and Oxygenates - Transformation   
country_or_area year                                              
Afghanistan     1990                                        NaN   
                1991                                        NaN   
                1992                                        NaN   

                                                                                   \
commodity_transaction Additives and Oxygenates - Transformation in oil refineries   
country_or_area year                                                                
Afghanistan     1990                                                 NaN            
                1991                                                 NaN            
                1992                                                 NaN            

                                                                                  \
commodity_transaction Additives and Oxygenates - transfers and recycled products   
country_or_area year                                                               
Afghanistan     1990                                                 NaN           
                1991                                                 NaN           
                1992                                                 NaN           

                                                                                  \
commodity_transaction Animal waste - Consumption by commerce and public services   
country_or_area year                             

In [10]:
# Unstack leaves you with an unpleasent nested/ multiindex column structure, which might have some utility I'm not aware of, but I'd rather remove for now at least
df_groupby.columns = df_groupby.columns.droplevel()

In [11]:
# We now have a dataframe which is easier to navigate, although ideally I'd like it not to be a groupby at all
df_groupby

commodity_transaction  Additives and Oxygenates - Exports  \
country_or_area year                                        
Afghanistan     1990                                  NaN   
                1991                                  NaN   
                1992                                  NaN   
                1993                                  NaN   
                1994                                  NaN   
...                                                   ...   
Zimbabwe        2010                                  NaN   
                2011                                  NaN   
                2012                                  NaN   
                2013                                  NaN   
                2014                                  NaN   

commodity_transaction  Additives and Oxygenates - Imports  \
country_or_area year                                        
Afghanistan     1990                                  NaN   
                1991                                  NaN   
                1992                                  NaN   
                1993                                  NaN   
                1994                                  NaN   
...                                                   ...   
Zimbabwe        2010                                  NaN   
                2011                                  NaN   
                2012                                  NaN   
                2013                                  NaN   
                2014                                  NaN   

commodity_transaction  Additives and Oxygenates - Production  \
country_or_area year                                           
Afghanistan     1990                                     NaN   
                1991                                     NaN   
                1992                                     NaN   
                1993                                     NaN   
                1994                                     NaN   
...                                                      ...   
Zimbabwe        2010                                     NaN   
                2011                                     NaN   
                2012                                     NaN   
                2013                                     NaN   
                2014                                     NaN   

commodity_transaction  Additives and Oxygenates - Receipts from other sources  \
country_or_area year                                                            
Afghanistan     1990                                                 NaN        
                1991                                                 NaN        
                1992                                                 NaN        
                1993                                                 NaN        
                1994                                                 NaN        
...                                                                  ...        
Zimbabwe        2010                                                 NaN        
                2011                                                 NaN        
                2012                                                 NaN        
                2013                                                 NaN        
                2014                                                 NaN        

commodity_transaction  Additives and Oxygenates - Stock changes  \
country_or_area year                                              
Afghanistan     1990                                        NaN   
                1991                                        NaN   
                1992                                        NaN   
                1993                                        NaN   
                1994                                        NaN   
...                                                     

In [12]:
# Would ideally like the index to not be a multiindex either
df_groupby.index

MultiIndex([('Afghanistan', 1990),
            ('Afghanistan', 1991),
            ('Afghanistan', 1992),
            ('Afghanistan', 1993),
            ('Afghanistan', 1994),
            ('Afghanistan', 1995),
            ('Afghanistan', 1996),
            ('Afghanistan', 1997),
            ('Afghanistan', 1998),
            ('Afghanistan', 1999),
            ...
            (   'Zimbabwe', 2005),
            (   'Zimbabwe', 2006),
            (   'Zimbabwe', 2007),
            (   'Zimbabwe', 2008),
            (   'Zimbabwe', 2009),
            (   'Zimbabwe', 2010),
            (   'Zimbabwe', 2011),
            (   'Zimbabwe', 2012),
            (   'Zimbabwe', 2013),
            (   'Zimbabwe', 2014)],
           names=['country_or_area', 'year'], length=5568)

In [13]:
# Apparently this will turn the multiindex into normal columns:
df_groupby.reset_index(inplace=True)

In [14]:
#df_groupby.drop('commodity_transaction',axis=1,inplace=True)
df_groupby.set_index('country_or_area',inplace=True)

The dataframe is now in a nice & clean (flat?) format for investigating as normal. 

This also seems to me like it's ready to be put into a SQL table, which will be a cool project. If we can get this data hosted in a SQL server, we can then connect it to a website. Using PHP and JavaScript, you could have user dropdowns to select for example country x in year y, and then return those values in a table, or maybe even make a live graph.

In [15]:
df_groupby

commodity_transaction,year,Additives and Oxygenates - Exports,Additives and Oxygenates - Imports,Additives and Oxygenates - Production,Additives and Oxygenates - Receipts from other sources,Additives and Oxygenates - Stock changes,Additives and Oxygenates - Total energy supply,Additives and Oxygenates - Transformation,Additives and Oxygenates - Transformation in oil refineries,Additives and Oxygenates - transfers and recycled products,...,White spirit and special boiling point industrial spirits - Transfers and recycled products,White spirit and special boiling point industrial spirits - Transformation,White spirit and special boiling point industrial spirits - Transformation in petrochemical plants,White spirit and special boiling point industrial spirits - consumption by other industries and construction,White spirit and special boiling point industrial spirits - final consumption,White spirit and special boiling point industrial spirits - production from plants,White spirit and special boiling point industrial spirits - production from refineries,Wind – Autoproducer,Wind – Main activity,animal waste - Transformation
country_or_area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
Afghanistan,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
